<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="800" height="300"></center>

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [11]:
import pandas as pd
import numpy as np

# Read bank_marketing csv file
df = pd.read_csv('bank_marketing.csv')
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [12]:
# Process the column 'last_contact_date'. Due to lack of column 'date', I transformed the 'day_of_week' instead
df['month'] = df['month'].replace({'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12})
df['day_of_week'] = df['day_of_week'].replace({'mon':2,'tue':3,'wed':4,'thu':5,'fri':6,'sat':7,'sun':0})
df['last_contact_date'] = '2022' + '-' + df['month'].astype(str) + '-' + df['day_of_week'].astype(str)
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,last_contact_date
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,5,2,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022-5-2
1,1,57,services,married,high.school,unknown,no,no,telephone,5,2,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022-5-2
2,2,37,services,married,high.school,no,yes,no,telephone,5,2,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022-5-2
3,3,40,admin.,married,basic.6y,no,no,no,telephone,5,2,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022-5-2
4,4,56,services,married,high.school,no,no,yes,telephone,5,2,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022-5-2


In [13]:
# Split & rename client table
client = df[["client_id","age","job", "marital", "education", "credit_default", "housing", "loan"]]
client_rename = {'client_id':'id'}
client.rename(columns = client_rename,inplace=True)
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [14]:
# Split & rename campaign table
campaign = df[["client_id", "duration", "pdays", "previous", "y","poutcome", "campaign","last_contact_date"]]
campaign_rename = {'duration':'contact_duration','previous':'previous_campaign_contacts','y':'campaign_outcome','poutcome':'previous_outcome','campaign':'number_contacts'}
campaign.rename(columns = campaign_rename,inplace=True)
campaign.head()

,client_id,contact_duration,pdays,previous_campaign_contacts,campaign_outcome,previous_outcome,number_contacts,last_contact_date
0,0,261,999,0,no,nonexistent,1,2022-5-2
1,1,149,999,0,no,nonexistent,1,2022-5-2
2,2,226,999,0,no,nonexistent,1,2022-5-2
3,3,151,999,0,no,nonexistent,1,2022-5-2
4,4,307,999,0,no,nonexistent,1,2022-5-2


In [15]:
# Split & rename economics table
economics = df[["client_id", "emp_var_rate","cons_price_idx","euribor3m","nr_employed"]]
economics_rename = {'euribor3m':'euribor_three_months','nr_employed':'number_employed'}
economics.rename(columns = economics_rename,inplace=True)
economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [16]:
# Transform client table
education_replacement = {'.':'_','unknown':np.nan}
client['education'] = client['education'].str.replace('.','_')
client['education'] = client['education'].replace('unknown',np.nan)
client['job'] = client['job'].str.replace('.','')
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [17]:
# Transform campaign table
campaign['previous_outcome'] = campaign.previous_outcome.map({'success': 1, 'failure': 0,'nonexistent':np.nan})
campaign['campaign_outcome'] = campaign.campaign_outcome.map({'yes': 1, 'no': 0})
campaign['campaign_id'] = 1
campaign.head()

,client_id,contact_duration,pdays,previous_campaign_contacts,campaign_outcome,previous_outcome,number_contacts,last_contact_date,campaign_id
0,0,261,999,0,0,NaN,1,2022-5-2,1
1,1,149,999,0,0,NaN,1,2022-5-2,1
2,2,226,999,0,0,NaN,1,2022-5-2,1
3,3,151,999,0,0,NaN,1,2022-5-2,1
4,4,307,999,0,0,NaN,1,2022-5-2,1


In [18]:
# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

In [19]:
# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""